In [1]:

import warnings
warnings.filterwarnings('ignore')
import datetime
from torch.utils.data import Dataset
from skimage.transform import resize

from zipfile import ZipFile
import os
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from Dataset import FaceDataset,Scale,ToTensor
from Models import resblock,network_9layers,network_29layers,network_29layers_v2,save_model
from losses import batch_hard_triplet_loss
import skimage
from skimage.io import imread,imsave
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm 
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import os
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


ImportError: cannot import name '_validate_lengths'

In [2]:
!nvidia-smi

Fri Dec 27 14:58:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:B2:00.0 Off |                    0 |
| N/A   34C    P0    43W / 300W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from platform import python_version

print(python_version())

3.6.7


In [ ]:
df=pd.read_csv('image_data.csv')

# Light CNN


In [ ]:
msk = np.random.rand(len(df)) < 0.8

df_train=df[msk]
df_test=df[~msk]
print(len(df),'\n',len(df_train),'\n',len(df_test))
df_train.to_csv('train.csv')
df_test.to_csv('test.csv')

In [ ]:
os.mkdir('train/')
os.mkdir('test/')
for label in tqdm(df['Label'].unique()):
    os.mkdir('train/'+str(label))
    os.mkdir('test/'+str(label))
    

In [6]:
transform=torchvision.transforms.Compose([Scale(),ToTensor()])

train_dataset=FaceDataset('img_align_celeba','train.csv',transform)
train_loader=DataLoader(train_dataset,batch_size=8,shuffle=True)
test_dataset=FaceDataset('img_align_celeba','test.csv',transform)
test_loader=DataLoader(test_dataset,batch_size=8,shuffle=True)
print(test_loader.__len__())
print(train_loader.__len__())

25325
25325


In [ ]:
def visualize_loader(loader):
    for i,sample in enumerate(loader):
        #print(sample['image'].shape)
        if i==1:
            anchor_image, pos_image,neg_image= sample['anc']\
            , sample['pos'],sample['neg']
            
            print(anchor_image.shape)
            
#             anchor_image=np.squeeze(anchor_image.transpose(1,2,0),axis=2)
#             pos_image=np.squeeze(pos_image.transpose(1,2,0),axis=2)
#             neg_image=np.squeeze(neg_image.transpose(1,2,0),axis=2)
            
            
#             #print(image)
#             fig=plt.figure()
#             plt.imshow(anchor_image)
#             fig1=plt.figure()
#             plt.imshow(pos_image)
#             fig2=plt.figure()
#             plt.imshow(neg_image)
           
            break
visualize_loader(test_loader)

In [ ]:
block=resblock
model=network_29layers_v2(block, [1, 2, 3, 4])

# model_start_date=datetime.datetime.now().strftime("%Y_%m_%d")
# BEST_MODEL_PATH=os.path.join(os.getcwd(),'model_{}'.format(model_start_date))
# if not os.path.exists(BEST_MODEL_PATH):
#     os.mkdir(BEST_MODEL_PATH)
#     print('model_{} dir has been made'.format(model_start_date))
# print("Model's state_dict:")
# writer = SummaryWriter('model_{}/dapi_seg_experiment_{}'.format(model_start_date,1))
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())


# writer.close()

In [ ]:
# print(model(torch.ones((8,1,128,128))).shape)
import tensorflow as tf
a=tf.ones((8,8))
b=tf.linalg.diag_part(a)
print(b.shape,b)

In [ ]:

b1=tf.expand_dims(b, 0)
print(b1.shape)
b2=tf.expand_dims(b, 1)
print(b2.shape)

In [ ]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))
optimizer_selected='adam'
batchsize=4
no_steps=train_dataset.__len__()//batchsize
restart_epochs=8
num_epochs=10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


criterion=TripletLoss()

model = model.to(device)



if optimizer_selected=='adam':
    optimizer = torch.optim.Adam(model.parameters(),lr=10e-03, betas=(0.9, 0.98),weight_decay=0.02)
else:
    optimizer = torch.optim.SGD(model.parameters(),lr=10e-03, momentum=0.8,nesterov=True)

scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, restart_epochs*no_steps,\
                                                     eta_min=10e-012, last_epoch=-1)

best_val=10e06
for epoch in range(num_epochs):
    loop=tqdm(train_loader)
    print("Learning Rate : {}".format(optimizer.state_dict()['param_groups'][-1]['lr']))
    # loop over the dataset multiple times
    

    for mode in ['train','eval']:
     
        if mode == 'train':
            
            running_loss = 0.0
            model.train()

            for i, sample_batched in (enumerate(loop)):
                loop.set_description('Epoch {}/{}'.format(epoch + 1, num_epochs))
                #print("epoch {},iter {}".format(epoch,i))

                # zero the parameter gradients

                optimizer.zero_grad()
                # get the inputs; data is a list of [inputs, labels]
                anc_batch, pos_batch,neg_batch = sample_batched['anc'],sample_batched['pos'],sample_batched['neg']
                
                anc_batch, pos_batch,neg_batch = anc_batch.to(device, dtype = torch.float)\
                ,pos_batch.to(device, dtype = torch.float),neg_batch.to(device,dtype=torch.float)

                # forward + backward + optimize
                anc_embeddings=model(anc_batch)
                pos_embeddings=model(pos_batch)
                neg_embeddings=model(neg_batch)
                
                #outputs = torch.sigmoid()
                
                loss = criterion(anc_embeddings,pos_embeddings,neg_embeddings)
        
                loss.backward()
                optimizer.step()
                
                scheduler.step()

                    
                loop.set_postfix(train_loss=loss.detach().item())
                
                 
                    
        elif mode =='eval':
            optimizer.zero_grad()
            samples_test=len(test_loader)
            model.eval()
            final_test_loss=0
            
            for j, test_sample in enumerate(test_loader):
                
                optimizer.zero_grad()
                # get the inputs; data is a list of [inputs, labels]
                anc_batch_test, pos_batch_test,neg_batch_test = test_sample['anc'],test_sample['pos'],test_sample['neg']
                
                anc_batch_test, pos_batch_test,neg_batch_test = anc_batch_test.to(device, dtype = torch.float)\
                ,pos_batch_test.to(device, dtype = torch.float),neg_batch_test.to(device,dtype=torch.float)

                # forward + backward + optimize
                anc_embeddings_test=model(anc_batch_test)
                pos_embeddings_test=model(pos_batch_test)
                neg_embeddings_test=model(neg_batch_test)
                
                #outputs = torch.sigmoid()
                
                test_loss = criterion(anc_embeddings_test,pos_embeddings_test,neg_embeddings_test)
        
                
                final_test_loss+=test_loss.detach().item()
                
                if j%100==99:
                    writer.add_scalar('testing loss',test_loss.detach().item(),epoch * len(test_loader) + j)
                
            print("final test_loss: {}".format(final_test_loss/(j+1)))
            
            if final_test_loss/(j+1)<best_val:
                best_val=final_test_loss/(j+1)
                name=BEST_MODEL_PATH+'/model_optim.pth'
                save_model(model,optimizer,name,scheduler=None)
                print("saved model with test dice score: {}".format(best_val))

name=BEST_MODEL_PATH+'/model_final.pth'
save_model(model,optimizer,name,scheduler=None)
        
        

print('Finished Training')



In [ ]:
z=b1-2*a+b2
print(z.shape,z)

# M2GAN

In [ ]:
class conv_block(nn.Module):
    def __init__(self,channel_in,channel_out,kernel_size,padding,stride):
        super().__init__()
        self.conv=nn.Sequential(nn.Conv2d(channel_in,channel_out,kernel_size=kernel_size,stride=stride,padding=padding,bias=True),
                                nn.BatchNorm2d(channel_out),
                                nn.ReLU(inplace=True))
    
    def forward(self,x):
        x=self.conv(x)
        return x

# class de_conv


In [ ]:
!nvidia-smi